# Chapter 9: Population Methods

In [3]:
import numpy as np
from scipy import stats

## Algorithm 9.1

In [7]:
def rand_population_uniform(m, a, b):
    d = a.shape[0]
    return [ a+np.random.random(d)*(b-a) for i in range(m) ]

## Algorithm 9.2

In [11]:
def rand_population_normal(m ,mu, sigma):
    D = stats.multivariate_normal(mean=mu, cov=sigma)
    return [ D.rvs() for i in range(m)]

## Algorithm 9.3

In [13]:
def rand_population_cauchy(m, mu, sigma):
    n = mu.shape[0]
    return [ [stats.cauchy(loc=mu[i],scale=sigma[i]).rvs() for i in range(n)] for j in range(m)]

## Algorithm 9.5

In [40]:
def rand_population_binary(m, n):
    nums = []
    for i in range(m):
        arr = np.random.randint(2, size=(n,))
        num = str(arr).replace('[','').replace(']','').replace(' ','')
        nums.append(num)
    return nums

## Algorithm 9.6

In [41]:
def TruncationSelection(k, y):
    p = np.argsort(y)
    return [ p[np.random.randint(k,size=(2,))] for i in y ]

def TournamentSelection(k, y):
    def getparent():
        p = np.random.permutation(len(y))
        y_np = np.array(y)
        return np.argmin(y_np[p[:k]])
    return [ [getparent(), getparent()] for i in y ]

def RouletteWheelSelection(y):
    y = np.max(y) - np.array(y)
    normalize = y / np.linalg.norm(y, 1)
    return [np.random.choice(a=np.arange(0,len(y),1), p=normalize, size=2 ) for i in y]

## Algorithm 9.7

In [42]:
'''
For Binary string chromosomes
'''
def singlePointCrossover(a,b):
    i = np.random.randint(low=0, high=len(a))
    return a[:i] + b[i:]

def twoPointCrossover(a, b):
    n = len(a)
    i, j = np.random.randint(low=0,high=n,size=2)

    if i > j:
        (i, j) = (j,i)
    
    return a[:i] + b[i:j] + a[j:n]

def uniformCrossover(a, b):
    child = copy(a)
    for i in range(len(a)):
        if np.random.random() < 0.5:
            child[i] = b[i]
    
    return child